## Overview of the Course

In [2]:
import pandas as pd
import numpy as np

# for handling text
import nltk
%matplotlib inline
import sklearn
import regex as re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
from termcolor import colored

# for plots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
%matplotlib inline

# for model creation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# we specify for the columns to be shown all together without truncation
pd.set_option('display.max_columns', None)

# Row
pd.set_option('display.max_rows', 10)

# path to the data
ALLData_FILE_PATH = r"D:\Dr. Desh\REConference\Tutorial\3 Preprocessing - Mitra\AllData.csv"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the file

In [3]:
all_data = pd.read_csv(ALLData_FILE_PATH)
all_data.head(2)

c:\Users\mitra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (7,8,9,10,11,13,16,18,19,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,70,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,96,98,99,101,102,103,104,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,139,140,145,146,147,148,149,150,151,152,153,154,155,156,169) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,alias,assigned_to,assigned_to_detail,blocks,cc,cc_detail,cf_backlog,cf_blocking_20,cf_blocking_fennec,cf_blocking_seamonkey21,cf_blocking_thunderbird31,cf_blocking_webextensions,cf_bmo_component_move,cf_cab_review,cf_colo_site,cf_crash_signature,cf_data_science_status,cf_due_date,cf_firefox_messaging_system,cf_fission_milestone,cf_fx_iteration,cf_fx_points,cf_has_regression_range,cf_has_str,cf_last_resolved,cf_locale,cf_platform_rel,cf_qa_whiteboard,cf_rank,cf_status_conduit_push,cf_status_firefox25,cf_status_firefox26,cf_status_firefox27,cf_status_firefox28,cf_status_firefox29,cf_status_firefox33,cf_status_firefox34,cf_status_firefox40,cf_status_firefox41,cf_status_firefox42,cf_status_firefox43,cf_status_firefox44,cf_status_firefox45,cf_status_firefox46,cf_status_firefox47,cf_status_firefox48,cf_status_firefox49,cf_status_firefox50,cf_status_firefox51,cf_status_firefox52,cf_status_firefox53,cf_status_firefox54,cf_status_firefox55,cf_status_firefox56,cf_status_firefox57,cf_status_firefox58,cf_status_firefox59,cf_status_firefox60,cf_status_firefox61,cf_status_firefox62,cf_status_firefox63,cf_status_firefox64,cf_status_firefox65,cf_status_firefox66,cf_status_firefox67,cf_status_firefox67_0_1,cf_status_firefox68,cf_status_firefox69,cf_status_firefox70,cf_status_firefox_esr52,cf_status_firefox_esr60,cf_status_firefox_esr68,cf_status_fxios_v11_0,cf_status_fxios_v11_1,cf_status_fxios_v12_0,cf_status_fxios_v12_1,cf_status_fxios_v12_2,cf_status_fxios_v13_0,cf_status_fxios_v13_1,cf_status_fxios_v13_2,cf_status_fxios_v13_3,cf_status_fxios_v14_0,cf_status_fxios_v14_1,cf_status_fxios_v14_2,cf_status_fxios_v14_3,cf_status_fxios_v15_0,cf_status_fxios_v15_1,cf_status_fxios_v16_0,cf_status_geckoview62,cf_status_geckoview66,cf_status_seamonkey21,cf_status_seamonkey210,cf_status_seamonkey211,cf_status_seamonkey249,cf_status_seamonkey253,cf_status_seamonkey256,cf_status_seamonkey257esr,cf_status_seamonkey258,cf_status_seamonkey259,cf_status_seamonkey263,cf_status_thunderbird_59,cf_status_thunderbird_60,cf_status_thunderbird_61,cf_status_thunderbird_62,cf_status_thunderbird_63,cf_status_thunderbird_64,cf_status_thunderbird_65,cf_status_thunderbird_66,cf_status_thunderbird_67,cf_status_thunderbird_68,cf_status_thunderbird_69,cf_status_thunderbird_70,cf_status_thunderbird_esr60,cf_status_thunderbird_esr68,cf_tracking_bmo_push,cf_tracking_conduit_push,cf_tracking_e10s,cf_tracking_firefox30,cf_tracking_firefox31,cf_tracking_firefox45,cf_tracking_firefox46,cf_tracking_firefox47,cf_tracking_firefox50,cf_tracking_firefox51,cf_tracking_firefox52,cf_tracking_firefox53,cf_tracking_firefox61,cf_tracking_firefox63,cf_tracking_firefox64,cf_tracking_firefox65,cf_tracking_firefox66,cf_tracking_firefox67,cf_tracking_firefox68,cf_tracking_firefox69,cf_tracking_firefox70,cf_tracking_firefox_esr60,cf_tracking_firefox_esr68,cf_tracking_firefox_relnote,cf_tracking_fxios,cf_tracking_geckoview62,cf_tracking_seamonkey249,cf_tracking_seamonkey253,cf_tracking_seamonkey257esr,cf_tracking_seamonkey263,cf_tracking_thunderbird_59,cf_tracking_thunderbird_60,cf_tracking_thunderbird_61,cf_tracking_thunderbird_62,cf_tracking_thunderbird_63,cf_tracking_thunderbird_64,cf_tracking_thunderbird_65,cf_tracking_thunderbird_66,cf_tracking_thunderbird_67,cf_tracking_thunderbird_68,cf_tracking_thunderbird_69,cf_tracking_thunderbird_70,cf_tracking_thunderbird_esr60,cf_tracking_thunderbird_esr68,cf_user_story,cf_webcompat_priority,classification,comment_count,component,creation_time,creator,creator_detail,depends_on,dupe_of,duplicates,flags,groups,id,is_cc_accessible,is_confirmed,is_creator_accessible,is_open,keywords,last_change_time,mentors,mentors_detail,op_sys,platform,priority,product,qa_contact,qa_contact_detail,regressed_by,regressions,resolution,see_also,severity,status,summary,target_milestone,type,url,version,votes,whiteboard
0,0,NaN,nobody@mozilla.org,"{'email': 'nobody@mozilla.org', 'real_name': '...","10276, 104166","['amanforindia@gmail.com', 'andrewm715+bugzill..."

In [4]:
# remove the redundant column
all_data.drop(columns=['Unnamed: 0'], inplace=True)

# drop duplicate ids
all_data.drop_duplicates(subset='id', inplace=True)

# drop columns that only contain null
all_data.dropna(axis=1, inplace=True, how='all')

### Data Exploration

First, we take a look at the data frame size. Then we realize that we don't need all the 198 columns, so we choose only the ones that can be helpful in understanding the data.

However, it is always necessary to look at the documentation for the data from the source website to understand it better

In [5]:
print(f'There are {len(all_data)} records and {len(all_data.columns)} columns in this dataset.')

There are 4215 records and 198 columns in this dataset.


In [6]:
df = all_data[[
        'classification', 'component', 'platform',  'assigned_to', 
        'summary', 'id', 'status', 'is_confirmed', 'votes',
        'severity', 'keywords', 'priority', 'creator', 'type'
        ]]

In [7]:
print(f'There are {len(df)} records and {len(df.columns)} columns in this dataset.')

There are 4215 records and 14 columns in this dataset.


### Visualize the data to understand it better

In the first part we try to categorize the data into groups and count their frequency. These categories will be placed in a data frame along with the number of times each one was seen.

Here is an overview of what the functions in the next cells will do:


In [8]:
# choose a column
col = 'platform'

# create a new dataset by grouping the categories, then choose id to show the count
categories_count = df.groupby(by=col).count().id.sort_values(ascending=False)

# by resetting the index, we create a df
categories_count_df = categories_count.reset_index()

categories_count_df

,platform,id
0,Unspecified,2727
1,All,1228
2,x86,138
3,x86_64,45
4,ARM64,26
5,Other,18
6,Desktop,14
7,ARM,10
8,PowerPC,8
9,HP,1


In [9]:
categories_count_df.rename(columns={'id': 'count'}, inplace=True)
categories_count_df

,platform,count
0,Unspecified,2727
1,All,1228
2,x86,138
3,x86_64,45
4,ARM64,26
5,Other,18
6,Desktop,14
7,ARM,10
8,PowerPC,8
9,HP,1


In [10]:
def create_grouped_df(df, col):
    """
    creates a dataset for counting categories in one field of a df
    """
    series = df.groupby(by=col).count().id.sort_values(ascending=False).reset_index()
    df_ = pd.DataFrame(series)
    df_.rename(columns={'id': 'count'}, inplace=True)

    return df_

In [11]:
def create_plot(df, col):

    # create the dataset for the visualization
    df_ = create_grouped_df(df, col=col)

    # start plotting
    fig = px.bar(df_, x=col, y='count', text='count',
     labels={'x': col, 'y': 'frequency'}, color='count',
     color_continuous_scale='Agsunset_r')

    fig.update_traces(
        texttemplate='%{text:.2f}', textposition='outside',
        textfont_size=10)

    # Change the bar mode
    fig.update_layout(title_text=f'Field {col} types count')
    fig.show()

Many fields have more than 20 categories which can make visualization and depicting the values challenging. To prevent the plot from getting too messy and illegible, we filter the columns with less than or equal to 20 categories.

In [12]:
# a list of columns we want to visalize
visualization_cols = []

# start filtering
for col in df.columns:
    unique_values = df.loc[:, col].unique()

    if len(unique_values)<20 and len(unique_values)>1:
        print('column: ', colored(f'{col}', 'green'))
        print(unique_values)
        visualization_cols.append(col)

column:  classification
['Components' 'Client Software' 'Server Software' 'Other'
 'Developer Infrastructure']
column:  platform
['All' 'x86' 'PowerPC' 'Unspecified' 'x86_64' 'HP' 'ARM' 'Other' 'Desktop'
 'ARM64']
column:  status
['NEW' 'ASSIGNED' 'REOPENED' 'UNCONFIRMED' 'RESOLVED']
column:  is_confirmed
[ True False]
column:  severity
['normal' 'critical' 'minor' 'blocker' 'major' 'trivial']
column:  priority
['P3' '--' 'P5' 'P2' 'P4' 'P1']
column:  type
['enhancement' 'task']


In [13]:
# loop over the columns and visualize them
for col in visualization_cols:
    create_plot(df, col=col)

### Working with textual data

We'll start by looking at the summary column. As we discussed earlier, first, we'll tokenize the text for a more accurate cleaning, then, based on our task, we choose a couple of cleaning functions one after another. These are the functions:

 1. Lowercase the text
 2. remove the numbers
 3. Remove the stopwords
 4. Remove the punctuation

When we go over all the steps, we append the text to a new list and then attach it to the main dataset.



In [14]:

def tokenize_text(df, input_col='summary', output_col="summary_tokens_list"):
    """
    takes a dataset and name of a column
    then tokenizes the text in the column of that dataset
    """
    tokenized_series = df.loc[:, input_col].apply(lambda t: word_tokenize(t))
    df.loc[:, output_col] = tokenized_series
    return df
    
df = tokenize_text(df)

c:\Users\mitra\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
df.head(2)

,classification,component,platform,assigned_to,summary,id,status,is_confirmed,votes,severity,keywords,priority,creator,type,summary_tokens_list,cleaned_word_list
0,Components,Networking: HTTP,All,nobody@mozilla.org,Proxy: map HTTP 500 errors to necko errors (so...,2875,NEW,True,13,normal,[],P3,erik@vanderpoel.org,enhancement,"[Proxy, :, map, HTTP, 500, errors, to, necko, ...","[proxy, map, http, errors, necko, errors, inte..."
1,Components,LDAP Integration,All,nobody@mozilla.org,[LDAP] Access to a local LDAP server in Off-Li...,2892,NEW,True,4,normal,['helpwanted'],--,lchiang@formerly-netscape.com.tld,enhancement,"[[, LDAP, ], Access, to, a, local, LDAP, serve...","[ldap, access, local, ldap, server, off-line, ..."


In [15]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(list_of_words):
    filtered_list = [w for w in list_of_words if not w.lower() in stop_words]
    return filtered_list

def check_punct(list_of_words):
    """
    look at the tokenized text. if the token was matched with ( or was only)
    a puncutation, it is redundant and we remove it.
    """
    filtered_list = []
    for word in list_of_words:
        if re.match("[()!><.,`?':\-\[\]_@]", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list

def remove_numbers(list_of_words):

    filtered_list = []
    for word in list_of_words:
        if re.findall("[0-9]", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list


def clean(df, input_col='summary_tokens_list', output_col="cleaned_word_list"):
    
    """
    takes a column of textual data 
    outputs a df with cleaned text attached
    """
    text_series = df.loc[:, input_col]
    cleaned_text_series = []
    
    for text in text_series: 
        # go through every record of this dataset
        # then append the cleaned text to a new list
        text = [t.lower() for t in text]
        cleaned_text = remove_numbers(text)
        cleaned_text = remove_stopwords(cleaned_text)
        cleaned_text = check_punct(cleaned_text)
        cleaned_text_series.append(cleaned_text)
        
    df.loc[:, output_col] = cleaned_text_series
    
    return df

df = clean(df)

In [17]:
df.head(2)

,classification,component,platform,assigned_to,summary,id,status,is_confirmed,votes,severity,keywords,priority,creator,type,summary_tokens_list,cleaned_word_list
0,Components,Networking: HTTP,All,nobody@mozilla.org,Proxy: map HTTP 500 errors to necko errors (so...,2875,NEW,True,13,normal,[],P3,erik@vanderpoel.org,enhancement,"[Proxy, :, map, HTTP, 500, errors, to, necko, ...","[proxy, map, http, errors, necko, errors, inte..."
1,Components,LDAP Integration,All,nobody@mozilla.org,[LDAP] Access to a local LDAP server in Off-Li...,2892,NEW,True,4,normal,['helpwanted'],--,lchiang@formerly-netscape.com.tld,enhancement,"[[, LDAP, ], Access, to, a, local, LDAP, serve...","[ldap, access, local, ldap, server, off-line, ..."
